### Testing the Network which generates the embeddings

In [1]:
import sys
sys.path.append("../")

import random
import torch
import matplotlib.pyplot as plt

from backbone.dataset import Face_Dataset_Triplet
from backbone.loss import TripletLoss

from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch import optim
import torch.nn as nn

Using TensorFlow backend.


In [2]:
face_data = Face_Dataset_Triplet(root_dir= "../lfw/", file_root = "../files/", train=True)

In [3]:
from backbone.networks.inception_resnet_v1 import InceptionResnetV1

In [4]:
Siamese_network = InceptionResnetV1(pretrained='vggface2')
checkpoint = torch.load("../pretrained/20180402-114759-vggface2.pt")
Siamese_network.load_state_dict(checkpoint)
Siamese_network = Siamese_network.cuda()

In [5]:
face_data_test = Face_Dataset_Triplet(root_dir= "../lfw/", 
                              file_root = "../files/", 
                              test=True)

test_dataloader = DataLoader(face_data_test,
                        shuffle=True,
                        num_workers=8,
                        batch_size=1)

#### Verify the shape of embeddings

In [6]:
for data in test_dataloader:
    break
    
img = data[0].cuda()
embeddings = Siamese_network(img)

print(img.squeeze(0).shape,embeddings.shape)

torch.Size([3, 250, 250]) torch.Size([1, 512])


In [11]:
criterion = TripletLoss(margin = 1.)

img1 = data[0].cuda()
img2 = data[1].cuda()
img3 = data[2].cuda()

fv1 = Siamese_network(img1)
fv2 = Siamese_network(img2)
fv3 = Siamese_network(img3)

loss = criterion(fv1,fv2,fv3)

print(loss)

tensor(1.0986, device='cuda:0', grad_fn=<MeanBackward0>)
